In [1]:
!pip install ultralytics 
!pip install lightning 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.1/949.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 18.0 MB/s eta 0:00:00


In [2]:
# Import necessary libraries

import warnings
warnings.filterwarnings("ignore")

# -----------------------
import numpy as np
import pandas as pd
from pathlib import Path
import os
import re
from typing import List
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import cv2

# PyTorch and related libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import v2

# YOLO and Timmy (for image-based models)
from ultralytics import YOLO
import timm

# Scikit-learn for data splitting
from sklearn.model_selection import train_test_split

# PyTorch Lightning for model training and callbacks
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
import lightning as L

# Copy for deep copy operations
import copy


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
def seed_everything(seed):
    torch.manual_seed(seed) 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False
    L.pytorch.seed_everything(seed, workers=True)
    
seed_everything(42)

INFO: Seed set to 42


## Variables

In [4]:
work_dir = "/kaggle/input/zindi-cgiar-root-volume-estimation-challenge"

TRAIN_DATA_PATH = Path(os.path.join(work_dir, "data/train"))
TEST_DATA_PATH = Path(os.path.join(work_dir, "data/test"))


train_df = pd.read_csv(os.path.join(work_dir,"Train.csv"))
test_df = pd.read_csv(os.path.join(work_dir,"Test.csv"))
submission_df = pd.read_csv(os.path.join(work_dir, "Sample_Submission.csv"))

In [5]:
debug_ = True

agg_data = 'mean'

bs_ = 16
workers_ = 0

model_name_ = 'focalnet_xlarge_fl3.ms_in22k' 

max_epochs_ = 20

img_width  =  384 
img_height =  384 

In [6]:

display(train_df.head())
display(test_df.head())
print('---'*10)
print(test_df.shape, train_df.shape, submission_df.shape)
print('---'*10)
print(train_df['Stage'].value_counts())
print(test_df['Stage'].value_counts())
print('---'*10)
print(train_df['FolderName'].unique().size, test_df['FolderName'].unique().size)

,ID,FolderName,PlantNumber,Side,Start,End,RootVolume,Genotype,Stage
0,ID_826322_Lbmaya,A6dzrkjqvl,3,L,33,42,0.9,IITA-TMS-IBA000070,Early
1,ID_718181_Bslpwx,Ypktwvqjbn,7,L,33,41,1.5,IBA154810,Early
2,ID_465762_L1n61d,Ox18ob0syv,4,R,21,28,2.7,IBA980581,Early
3,ID_626872_Pbmx2e,Hqcekwpxgu,2,R,30,39,2.6,IITA-TMS-IBA000070,Early
4,ID_518846_Opko8c,Ummqfuetoc,1,R,17,26,2.7,IBA980581,Early


,ID,FolderName,PlantNumber,Side,Start,End,Genotype,Stage
0,ID_208667_Hnkl8q,L5l1h3kekg,7,L,38,50,IITA-TMS-IBA000070,Early
1,ID_285249_Jnjvav,Wgutyon8uu,6,R,23,37,TMEB419,Early
2,ID_697947_Yec6bd,Mylwjeq6tq,3,R,19,42,IBA980581,Early
3,ID_534638_X3j91f,Pfp24vx905,2,R,27,34,TMEB419,Early
4,ID_929298_Xvymuz,Mrw7chmalv,4,R,30,43,IBA154810,Early


------------------------------
(130, 8) (386, 9) (130, 2)
------------------------------
Stage
Early    370
Late      16
Name: count, dtype: int64
Stage
Early    125
Late       5
Name: count, dtype: int64
------------------------------
98 62


In [7]:
if debug_:
    folder = "Ypktwvqjbn"
    df = train_df[train_df['FolderName'] == folder]

    df[['FolderName', 'PlantNumber', 'Side', 'Start', 'End', 'RootVolume']].head(n = 10)

## Load the segmentation models

In [8]:
# Since the organizers didn't provide performance metrics, we will be using all the models (For Now)
segmentation_models = {
    "full" : YOLO(os.path.join('/kaggle/input/cgiar-yolo-models/pytorch/default/1',"Models/best_full.pt")),
    "early" : YOLO(os.path.join('/kaggle/input/cgiar-yolo-models/pytorch/default/1',"Models/best_early.pt")),
    "late" : YOLO(os.path.join('/kaggle/input/cgiar-yolo-models/pytorch/default/1',"Models/best_late.pt"))
}

In [9]:
'''
Explanation
We are going to use get_segmented_images function to process a set of images and extracts the regions of interest (i.e., root segments) using a pre-trained segmentation models. It then returns the segmented images where roots are detected, or all images if no detections are made.
'''

def get_segmented_images(image_paths, display_image=False):
    """Extracts and merges segments from images, returning only images with detections."""

    for model in segmentation_models.keys():
        model = segmentation_models[model]
        results = model(image_paths, verbose=False)

        if len(results[0].boxes.xyxy) != 0:
            break

    if len(results[0].boxes.xyxy) == 0:
        # Incase of no detections, return all the images (Still thinking of better ways to overcome this)
        return [Image.open(img) for img in image_paths]
        
    segmented_images = []
    
    for img_path, result in zip(image_paths, results):
        original_image = Image.open(img_path)
        merged_image = Image.new("RGBA", original_image.size, (0, 0, 0, 0))
        
        # Skip if no detections
        if len(result.boxes.xyxy) == 0:
            continue 
            
        # Extract and paste segments
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box.tolist())
            segment = original_image.crop((x1, y1, x2, y2))
            merged_image.paste(segment, (x1, y1))
            
        segmented_images.append(merged_image)

    # Display Images
    if display_image and segmented_images:
        fig, axes = plt.subplots(1, len(segmented_images), figsize=(15, 10))
        if len(segmented_images) == 1:
            axes = [axes]
        for ax, img in zip(axes, segmented_images):
            ax.imshow(img)
            ax.axis("off")
        plt.show()

    return segmented_images



# The function retrieves images from a specified folder
# based on the scan side (left or right) and a layer range
# (from a starting layer to an ending layer).

def get_images_within_range(base_path: Path, folder: str, side: str, start: int, end: int) -> list[Path]:
    """
    Get images from a folder that match the specified side (L/R) and layer range.
    
    Args:
        base_path: Root directory containing all folders
        folder: Name of the target folder (e.g., 'Ypktwvqjbn')
        side: Scan side to filter ('L' or 'R')
        start: Starting layer (inclusive)
        end: Ending layer (inclusive)
    """
    folder_path = base_path / folder
    
    # Get all files in the folder
    try:
        images = os.listdir(folder_path)
    except FileNotFoundError:
        return []

    # Regex pattern to extract side and layer from filenames
    pattern = re.compile(r'_([LR])_(\d{3})\.png$')
    
    selected_images = []
    
    for img_name in images:
        match = pattern.search(img_name)
        if match:
            # Extract side and layer from filename
            img_side = match.group(1)
            layer = int(match.group(2))
            
            # Check if matches criteria
            if img_side == side and start <= layer <= end:
                selected_images.append(folder_path / img_name)
    
    return selected_images



# finally merge all of the image segments into one
def merge_segmented_images(path: Path, folder: str, side: str, start: int, end: int):
    images_in_range = get_images_within_range(path, folder, side, start, end)
    segmented_images = get_segmented_images(images_in_range)

    # Determine final dimensions for the merged image
    total_width = sum(img.width for img in segmented_images)  # Sum of all widths
    max_height = max(img.height for img in segmented_images)  # Max height among all images

    # Create a blank canvas with a transparent background
    merged_image = Image.new("RGBA", (total_width, max_height), (0, 0, 0, 0))

    # Paste each segmented image next to the previous one (left to right)
    x_offset = 0
    for img in segmented_images:
        merged_image.paste(img, (x_offset, 0), img)  # Paste at correct position
        x_offset += img.width  # Move x-offset to the right for the next image

    return merged_image

In [10]:
OUTPUT_IMAGE_DIR = Path(os.path.join('/kaggle/working/', "merged_images/"))
os.makedirs(OUTPUT_IMAGE_DIR, exist_ok = True)

TRAIN_OUTPUT_DIR = OUTPUT_IMAGE_DIR / "Train"
TEST_OUTPUT_DIR = OUTPUT_IMAGE_DIR / "Test"

os.makedirs(TRAIN_OUTPUT_DIR, exist_ok = True)
os.makedirs(TEST_OUTPUT_DIR, exist_ok = True)

## Generate New Dataset

In [11]:
# Iterate through all the  images in the df
def generate_merged_images(df: pd.DataFrame, output_path: Path, input_path: Path, start_ = None, end_ = None):
    gen_image_paths = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Merging Images"):
        if start_ is not None and end_ is not None:
            row["Start"] = start_
            row["End"] = end_
        else:
            row["Start"] = row["Start"]
            row["End"] = row["End"]


        gen_image = merge_segmented_images(
            path=input_path,
            folder=row["FolderName"],
            side=row["Side"],
            start=row["Start"],
            end=row["End"]
        )
        img_path = output_path / f"{row['ID']}.png"
        gen_image.save(img_path)
        gen_image_paths.append(img_path)

    df['image segments'] = gen_image_paths
    return df

def display_generated_images(image_paths):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    axes = axes.flatten()

    for i, img_path in enumerate(image_paths[:4]):  # Display first 4 images
        image = Image.open(img_path)
        axes[i].imshow(image)
        axes[i].axis("off")  # Hide axes
        axes[i].set_title(f"Image {i+1}")

    plt.tight_layout()
    plt.show()

In [12]:
try:
    new_train_df = pd.read_csv(os.path.join(work_dir, 'new_train_df.csv'))
    new_test_df = pd.read_csv(os.path.join(work_dir, 'new_test_df.csv'))
    
except:

    new_train_df = generate_merged_images(train_df, TRAIN_OUTPUT_DIR, TRAIN_DATA_PATH)
    new_test_df  = generate_merged_images(test_df, TEST_OUTPUT_DIR, TEST_DATA_PATH)

    new_train_df.to_csv(os.path.join('/kaggle/working/', 'new_train_df.csv'))
    new_test_df.to_csv(os.path.join('/kaggle/working/', 'new_test_df.csv'))


display(new_train_df.head(10))

Merging Images: 100%|██████████| 130/130 [00:31<00:00,  4.13it/s]


,ID,FolderName,PlantNumber,Side,Start,End,RootVolume,Genotype,Stage,image segments
0,ID_826322_Lbmaya,A6dzrkjqvl,3,L,33,42,0.9,IITA-TMS-IBA000070,Early,/kaggle/working/merged_images/Train/ID_826322_...
1,ID_718181_Bslpwx,Ypktwvqjbn,7,L,33,41,1.5,IBA154810,Early,/kaggle/working/merged_images/Train/ID_718181_...
2,ID_465762_L1n61d,Ox18ob0syv,4,R,21,28,2.7,IBA980581,Early,/kaggle/working/merged_images/Train/ID_465762_...
3,ID_626872_Pbmx2e,Hqcekwpxgu,2,R,30,39,2.6,IITA-TMS-IBA000070,Early,/kaggle/working/merged_images/Train/ID_626872_...
4,ID_518846_Opko8c,Ummqfuetoc,1,R,17,26,2.7,IBA980581,Early,/kaggle/working/merged_images/Train/ID_518846_...
5,ID_167189_Lxr1uc,B5myqsh1wi,4,R,26,35,1.3,TMEB419,Early,/kaggle/working/merged_images/Train/ID_167189_...
6,ID_171226_Jaqvha,L8w7zu7wek,5,R,27,33,4.5,IITA-TMS-IBA000070,Early,/kaggle/working/merged_images/Train/ID_171226_...
7,ID_174315_Lw9ftc,Izbgyxre0g,2,L,24,37,3.7,IBA154810,Early,/kaggle/working/merged_images/Train/ID_174315_...
8,ID_216014_Djn5n6,Vinlgebupo,2,L,23,29,0.4,IBA980581,Early,/kaggle/working/merged_images/Train/ID_216014_...
9,ID_737506_Hrpn4d,Hc3b9gicdo,5,L,37,45,3.1,IITA-TMS-IBA000070,Early,/kaggle/working/merged_images/Train/ID_737506_...


In [13]:
if agg_data != None:
    print("Aggregated Data With Method:", agg_data )
    new_train_df['RootVolume'] = new_train_df.groupby('FolderName')['RootVolume'].transform(agg_data) # 'mean'


Aggregated Data With Method: mean


In [14]:
# Display Train
#display_generated_images(new_train_df['image segments'].values)

## Selected Models: Feature Extraction Tab Models - V2

In [15]:
import os
import copy
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
import lightgbm as lgb
from sklearn.model_selection import train_test_split



In [16]:
class RootVolumeDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transform=None, is_train=True):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.is_train = is_train

    def __getitem__(self, index):
        image = Image.open(self.df['image segments'].iloc[index]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if self.is_train:
            label = self.df['RootVolume'].iloc[index]
            return image, torch.tensor(label, dtype=torch.float32)
        return image

    def __len__(self):
        return len(self.df)

# Assuming you already have train_df, new_train_df, and new_test_df from earlier processing:
train_df, val_df = train_test_split(new_train_df, test_size=0.2, random_state=42)

# Define your image transformations (example transforms)
train_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])
test_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])

# Create datasets and dataloaders
train_dataset = RootVolumeDataset(train_df, transform=train_transform, is_train=True)
val_dataset   = RootVolumeDataset(val_df, transform=test_transform, is_train=True)
test_dataset  = RootVolumeDataset(new_test_df, transform=test_transform, is_train=False)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [17]:
# =============================================================================
# 2. Image Feature Extraction via timm
# =============================================================================

# Create the timm model (set your model_name, e.g., "resnet50")
model_name = model_name_
feature_extractor = timm.create_model(model_name, pretrained=True, num_classes=0)
feature_extractor.eval()  # set model to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor.to(device)

def extract_features(dataloader, model, device):
    all_features = []
    all_targets = []  # Only used if targets are available.
    with torch.no_grad():
        for batch in dataloader:
            if isinstance(batch, (list, tuple)):
                images, targets = batch
                images = images.to(device)
                feats = model(images)
                all_features.append(feats.cpu().numpy())
                all_targets.append(targets.numpy())
            else:
                images = batch.to(device)
                feats = model(images)
                all_features.append(feats.cpu().numpy())
    features = np.concatenate(all_features, axis=0)
    if all_targets:
        targets = np.concatenate(all_targets, axis=0)
        return features, targets
    return features

print("Extracting training image features...")
X_train_img, y_train = extract_features(train_loader, feature_extractor, device)
print("Extracting validation image features...")
X_val_img, y_val = extract_features(val_loader, feature_extractor, device)
print("Extracting test image features...")
X_test_img = extract_features(test_loader, feature_extractor, device)

model.safetensors:   0%|          | 0.00/1.64G [00:00<?, ?B/s]

Extracting training image features...
Extracting validation image features...
Extracting test image features...


In [18]:
# =============================================================================
# 3. Process Tabular Data for LightGBM
# =============================================================================

def preprocess_tabular_features(df):
    # Copy and drop columns that are not useful for prediction
    df_proc = df.copy()
    drop_cols = ['ID', 'FolderName', 'image segments', 'RootVolume']  # identifiers and target
    df_proc.drop(columns=drop_cols, errors='ignore', inplace=True)
    
    # Ensure numeric columns are of the proper type
    df_proc['PlantNumber'] = pd.to_numeric(df_proc['PlantNumber'], errors='coerce')
    
    # One-hot encode categorical features: Side, Genotype, Stage
    df_proc = pd.get_dummies(df_proc, columns=['Side', 'Genotype', 'Stage'], drop_first=True)
    return df_proc

# Preprocess tabular features for train, validation and test
tab_train = preprocess_tabular_features(train_df)
tab_val   = preprocess_tabular_features(val_df)
tab_test  = preprocess_tabular_features(new_test_df)

# Make sure all dataframes have the same columns
tab_val = tab_val.reindex(columns=tab_train.columns, fill_value=0)
tab_test = tab_test.reindex(columns=tab_train.columns, fill_value=0)

# Convert to numpy arrays
X_train_tab = tab_train.to_numpy()
X_val_tab = tab_val.to_numpy()
X_test_tab = tab_test.to_numpy()

In [19]:
# =============================================================================
# 4. Combine Image and Tabular Features
# =============================================================================

X_train_combined = np.hstack([X_train_img, X_train_tab])
X_val_combined = np.hstack([X_val_img, X_val_tab])
X_test_combined = np.hstack([X_test_img, X_test_tab])

X_train_combined[0][-20:]

array([0.19018405675888062, -0.02936416305601597, 0.07069263607263565, 0.269324392080307, -0.19090519845485687, 0.4795982539653778, 0.12221825122833252, 0.40291181206703186, -0.11511332541704178, 5, 33, 42, False, False, False, False, False, False, True, False], dtype=object)

In [20]:
# =============================================================================
# 5. Train LightGBM Model
# =============================================================================

train_data = lgb.Dataset(X_train_combined, label=y_train)
val_data = lgb.Dataset(X_val_combined, label=y_val, reference=train_data)

In [21]:

# Define parameters.
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'learning_rate': 0.0125,


    'num_leaves': 31,
    'n_estimators': 62,
    'max_depth': 4,
    
    #'feature_fraction':0.40,
    #'bagging_fraction':0.60,
    #'is_unbalanced': True,
   
    
    'reg_sqrt': True,
   
    'extra_trees': True,
    

    #'stochastic_rounding': False,
    #'bagging_by_query': True,
    
    'drop_rate': 0.60,
    'path_smooth': 0.10,
    'use_missing': False,
    #'alpha': 0.95,


     'verbose': -1,
    'seed': 42,
    #'objective_seed': 53,
}

# Train the model with early stopping.
num_boost_round = 500
early_stopping_rounds = 50

print("Training LightGBM model...")
evals_result = {}
model_lgb = lgb.train(
    params,
    train_data,
    num_boost_round=num_boost_round,
    valid_sets=[train_data, val_data],
    valid_names=['train', 'valid'],
    callbacks=[
                lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=True),
            ],
    #early_stopping_rounds=early_stopping_rounds,
    #evals_result=evals_result,
    #verbose_eval=50
)

Training LightGBM model...
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[62]	train's rmse: 0.847848	valid's rmse: 0.953114


In [22]:
# =============================================================================
# 5. Predictions and Evaluation
# =============================================================================
def calculate_rmse(preds, targets):
    return np.sqrt(np.mean((preds - targets) ** 2))

# Make predictions on train and validation sets.
train_preds = model_lgb.predict(X_train_combined, num_iteration=model_lgb.best_iteration)
val_preds   = model_lgb.predict(X_val_combined, num_iteration=model_lgb.best_iteration)

train_rmse = calculate_rmse(train_preds, y_train)
val_rmse   = calculate_rmse(val_preds, y_val)
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")

print(f"RATIO: {train_rmse/val_rmse:.4f}")

Train RMSE: 0.8478
Validation RMSE: 0.9531
RATIO: 0.8896


In [23]:
# Get test predictions.
test_preds = model_lgb.predict(X_test_combined, num_iteration=model_lgb.best_iteration)

# Attach predictions to the test DataFrame.
new_test_df['RootVolume'] = test_preds
new_test_df.head()

,ID,FolderName,PlantNumber,Side,Start,End,Genotype,Stage,image segments,RootVolume
0,ID_208667_Hnkl8q,L5l1h3kekg,7,L,38,50,IITA-TMS-IBA000070,Early,/kaggle/working/merged_images/Test/ID_208667_H...,2.204191
1,ID_285249_Jnjvav,Wgutyon8uu,6,R,23,37,TMEB419,Early,/kaggle/working/merged_images/Test/ID_285249_J...,1.835494
2,ID_697947_Yec6bd,Mylwjeq6tq,3,R,19,42,IBA980581,Early,/kaggle/working/merged_images/Test/ID_697947_Y...,1.960487
3,ID_534638_X3j91f,Pfp24vx905,2,R,27,34,TMEB419,Early,/kaggle/working/merged_images/Test/ID_534638_X...,1.935600
4,ID_929298_Xvymuz,Mrw7chmalv,4,R,30,43,IBA154810,Early,/kaggle/working/merged_images/Test/ID_929298_X...,1.773414


In [ ]:

submission = new_test_df[['ID', 'RootVolume']]
submission.to_csv(os.path.join('/kaggle/working/', f"submission_TabularV3_LGBM_V2-1_Original.csv"), index = False)
submission.head()

,ID,RootVolume
0,ID_208667_Hnkl8q,2.204191
1,ID_285249_Jnjvav,1.835494
2,ID_697947_Yec6bd,1.960487
3,ID_534638_X3j91f,1.935600
4,ID_929298_Xvymuz,1.773414
